In [1]:
import sympy as sp

In [3]:
alpha = 1.2
x_0 = 0.1
r = [0.3, 0.35, 0.36]
W = [1, 0.7, 0.7]

In [4]:
# Define symbolic variables
m = 3  # Number of variables (you can change this)
u = sp.Matrix(sp.symbols(f'u1:{m+1}'))  # Creates a column vector [u1, u2, u3]
x = sp.symbols(f'x1:{m+1}')  # Creates symbols x1, x2, x3
lower_alpha = (alpha * sp.ones(3)).lower_triangular()
x_0 = sp.Matrix([x_0] * m)
r = sp.Matrix(r)

In [5]:
x_u = x_0 + lower_alpha * u
Ld = sum(W[i] * (r[i] - x_u[i]) ** 2 for i in range(m))

In [6]:
Ld.diff(u)

Matrix([
[6.912*u1 + 4.032*u2 + 2.016*u3 - 1.3368],
[4.032*u1 + 4.032*u2 + 2.016*u3 - 0.8568],
[2.016*u1 + 2.016*u2 + 2.016*u3 - 0.4368]])

In [7]:
upper_alpha = (alpha * sp.ones(3)).upper_triangular()
W_diag = sp.diag(W, unpack=True)
DuLd = 2 * upper_alpha * W_diag * (lower_alpha * u - r + x_0)
DuLd

Matrix([
[6.912*u1 + 4.032*u2 + 2.016*u3 - 1.3368],
[4.032*u1 + 4.032*u2 + 2.016*u3 - 0.8568],
[2.016*u1 + 2.016*u2 + 2.016*u3 - 0.4368]])

In [8]:
Lj = sum(W[i] * (u[i] - u[i-1]) ** 2 for i in range(1, m))

In [9]:
Lj.diff(u)

Matrix([
[          1.4*u1 - 1.4*u2],
[-1.4*u1 + 2.8*u2 - 1.4*u3],
[         -1.4*u2 + 1.4*u3]])

In [10]:
def DuLjMat(i, j):
    if i == j:
        if i == 0:
            return 2 * W[i + 1]
        elif i == m - 1:
            return 2 * W[i]
        else:
            return 2 * (W[i] + W[i + 1])
    elif i - 1 == j:
        return -2 * W[i]
    elif i + 1 == j:
        return -2 * W[i + 1] 
    else:
        return 0
    
Wj = sp.Matrix(3, 3, DuLjMat)

In [11]:
sol = (2 * upper_alpha * W_diag * lower_alpha + Wj).solve(2 * upper_alpha * W_diag * (r - x_0))
sol

Matrix([
[ 0.128436014068963],
[0.0724128683561799],
[0.0390124908458912]])

In [15]:
subs_dict = {u[i]: value for i, value in enumerate(sol)}
(Ld + Lj).diff(u).subs(subs_dict)

Matrix([
[1.94289029309402e-16],
[2.22044604925031e-16],
[5.55111512312578e-17]])

In [8]:
from typing import List, Tuple
import numpy as np

def optimize_mpc(alpha: float, x_0: float, u_0: float, n: int, plan: List[float], W_d: List[float], W_j: List[float]) -> Tuple[List[float], float]:
    '''
    Model:
        dx = alpha * du
        x: state (lataccel)
        u: input (steer command)

    Params:
        alpha: steer to lataccel coefficient
        x_0: initial lataccel
        u_0: last control
        n: size of prediction horizon, same as control horizon
        plan: lataccel targets for the length of prediction horizon
        W_d: Weights for deviation error
        W_j: Weights for jerk error (len n-1)
    
    Return:
        control: optimal control for the control horizon
        prediction: predicted future states following the optimal control
        obj: objective value for the optimal control
    '''

    # Solve for minima of objective function using closed form solution to gradient
    upper_alpha = np.triu(alpha * np.ones((n, n)))
    lower_alpha = np.tril(alpha * np.ones((n, n)))
    W_j = np.array(W_j)
    d1 = -2 * W_j # Off diagonal of DuJ
    d0 = np.zeros(n)
    d0[: -1] += 2 * W_j
    d0[1: ] += 2 * W_j
    DuJ = np.diag(d0) + np.diag(d1, 1) + np.diag(d1, -1)
    plan = np.array(plan)
    W_d = np.array(W_d)
    A = 2 * upper_alpha @ np.diag(W_d) @ lower_alpha + DuJ
    b = 2 * upper_alpha @ np.diag(W_d) @ (plan - x_0)
    control = np.linalg.solve(A, b)
    prediction = x_0 + lower_alpha @ control

    L_d = (W_d * (prediction - plan) ** 2).sum()
    L_j = (W_j * np.diff(control, n=1) **2).sum()

    return control, prediction, L_d, L_j
